In [ ]:
# Install ultralytics (YOLOv8)
!pip install ultralytics -q

# Verify installation
import ultralytics
ultralytics.checks()

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip dataset (upload yolo_dataset.zip to your Drive first)
!unzip -q /content/drive/MyDrive/yolo_dataset.zip -d /content/

# Verify structure
!ls -la /content/yolo_dataset/
!ls /content/yolo_dataset/images/

In [ ]:
# Visualize sample data with bounding boxes
import cv2
import matplotlib.pyplot as plt
import os
import random

def visualize_sample(dataset_path, split='train', n_samples=4):
    img_dir = os.path.join(dataset_path, 'images', split)
    lbl_dir = os.path.join(dataset_path, 'labels', split)
    
    images = os.listdir(img_dir)
    samples = random.sample(images, min(n_samples, len(images)))
    
    fig, axes = plt.subplots(1, n_samples, figsize=(16, 4))
    
    for ax, img_name in zip(axes, samples):
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        lbl_path = os.path.join(lbl_dir, img_name.replace('.png', '.txt'))
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                for line in f:
                    cls, cx, cy, bw, bh = map(float, line.strip().split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, 'enemy_uav', (x1, y1-5), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(img_name[:15])
    
    plt.tight_layout()
    plt.show()

visualize_sample('/content/yolo_dataset')

In [ ]:
# Train YOLOv8 model
from ultralytics import YOLO

# Load pretrained YOLOv8n (nano - fast inference for real-time tracking)
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data='/content/yolo_dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='simUAV_detector',
    patience=20,           # Early stopping
    save=True,
    plots=True,
    # Augmentations for better generalization
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15.0,
    translate=0.1,
    scale=0.5,
    flipud=0.0,            # No vertical flip (sky/ground matters)
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1
)

In [ ]:
# Evaluate model on test set
metrics = model.val(split='test')

print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")

In [ ]:
# Visualize training results
from IPython.display import Image, display

display(Image(filename='runs/detect/simUAV_detector/results.png', width=800))
display(Image(filename='runs/detect/simUAV_detector/confusion_matrix.png', width=600))

In [ ]:
# Run inference on test images
test_images = '/content/yolo_dataset/images/test/'

results = model.predict(
    source=test_images,
    save=True,
    conf=0.25,
    iou=0.45,
    show_labels=True,
    show_conf=True
)

# Display sample predictions
import glob

pred_images = glob.glob('runs/detect/predict/*.png')[:4]
fig, axes = plt.subplots(1, len(pred_images), figsize=(16, 4))

for ax, img_path in zip(axes, pred_images):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Export best.pt to Google Drive
!cp runs/detect/simUAV_detector/weights/best.pt /content/drive/MyDrive/best.pt

print("best.pt saved to Google Drive")
print("Next: Download best.pt and run airsim_tracker.py")

In [ ]:
# (Optional) Export to ONNX for faster inference
model.export(format='onnx', simplify=True)

!cp runs/detect/simUAV_detector/weights/best.onnx /content/drive/MyDrive/best.onnx
print("ONNX model exported")